# Capítulo III: Chains

### Ejemplo inicial en el uso de cadenas

Cada vez que se desee trabajar con una cadena, al menos, debe estar definido un prompt template e instanciado algún modelo de lenguaje

In [10]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [11]:
load_dotenv()

True

In [12]:
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="Cual es una buena actividad para realizar en {temporada}?",
            input_variables=["temporada"],
        )
    )

In [13]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)

In [14]:
print(chain.run("primavera"))



> Entering new LLMChain chain...
Prompt after formatting:
Human: Cual es una buena actividad para realizar en primavera?

> Finished chain.
Una buena actividad para realizar en primavera es hacer un picnic en un parque o jardín. Puedes llevar comida y bebidas, una manta para sentarte y disfrutar del buen clima. También puedes aprovechar para caminar y apreciar la belleza de las flores y árboles en floración.


In [15]:
#Las cadenas tambien pueden ser instanciadas con un estado de "memoria"
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory #<--- golazo que veremos más adelante

In [16]:
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory(),
    verbose=True
)

In [17]:
conversation.run("Dime los primeros 5 números de la serie de fibonacci")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Dime los primeros 5 números de la serie de fibonacci
AI:

> Finished chain.


'Los primeros 5 números de la serie de Fibonacci son 0, 1, 1, 2, y 3.'

In [18]:
conversation.run("¿Y cuál es el séptimo?") #<-- preserva el contexto de la pregunta anterior



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Dime los primeros 5 números de la serie de fibonacci
AI: Los primeros 5 números de la serie de Fibonacci son 0, 1, 1, 2, y 3.
Human: ¿Y cuál es el séptimo?
AI:

> Finished chain.


'El séptimo número de la serie de Fibonacci es 8.'

In [19]:
conversation.run("¿Y cual es el valor aproximado del cuociente entre el quinto y cuarto número de la secuencia?") #<-- preserva el contexto de la pregunta anterior



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Dime los primeros 5 números de la serie de fibonacci
AI: Los primeros 5 números de la serie de Fibonacci son 0, 1, 1, 2, y 3.
Human: ¿Y cuál es el séptimo?
AI: El séptimo número de la serie de Fibonacci es 8.
Human: ¿Y cual es el valor aproximado del cuociente entre el quinto y cuarto número de la secuencia?
AI:

> Finished chain.


'El cuarto número de la serie de Fibonacci es 2 y el quinto número es 3. Entonces, el cuociente entre el quinto y cuarto número sería 3/2, que es igual a 1.5.'

### Cadenas Secuenciales

Este tipo de cadenas agregan valor cuando el output de una es utilizado como input de la siguiente

In [21]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [22]:
#Esta cadena está relacionada al postulante a chef
llm = OpenAI(temperature=.7)
chef_template = """Actúa como un chef amateur. Yo te daré una temática y tu tarea será crear una receta en relación a esa temática.

Temática: {tematica}
Receta: Esta es la receta que has creado:"""
chef_prompt_template = PromptTemplate(input_variables=["tematica"], template=chef_template)
chef_chain = LLMChain(llm=llm, prompt=chef_prompt_template)

In [23]:
#Esta cadena está relacionada al experto evaluador de cocina
llm = OpenAI(temperature=.7)
template = """Actúa como experto chef de cocina. Eres parte del comité experto que entrega la medalla Michelin de cocina. Tu tarea será entregar una evaluación a la propuesta del postulante bajo criterios de creatividad, originalidad y sabor. 

Receta:
{receta}
Evaluación crítica del chef experto del comité de la medalla Michelin:"""
prompt_template = PromptTemplate(input_variables=["receta"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [24]:
#Esta es la cadena completa que toma en consideración las cadenas antes definidas
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chef_chain, review_chain], verbose=True)

In [25]:
review = overall_chain.run("Un plato inspirado en la tierra patagónica")



> Entering new SimpleSequentialChain chain...
 

Patagónico Carne Asada:

Ingredientes:

- 2kg de carne de ternera o de cordero
- 2 cebollas
- 2 dientes de ajo
- 1 pimiento rojo
- 2 cucharadas de aceite de oliva
- Sal y pimienta al gusto
- 2 cucharadas de orégano
- 2 cucharadas de tomillo
- 2 cucharadas de comino
- 2 cucharadas de romero
- 2 cucharadas de tomatera
- 1 cucharada de vinagre de manzana

Instrucciones:

1. Precaliente el horno a 200 grados.

2. En una sartén, a fuego medio, calienta el aceite de oliva y agrega la cebolla, el ajo y el pimiento. Saltea hasta que estén suaves.

3. Mientras tanto, en un plato hondo, mezcla todas las


Esta receta ofrece una gran combinación de sabores y texturas. Los ingredientes se mezclan para crear un sabor único y original. El uso de los diferentes tipos de hierbas y especias, junto con el vinagre de manzana, aporta un toque ácido que equilibra el sabor a la perfección. La carne es cocinada al punto y se mantienen los jugos y la suavidad

Asimismo, es posible incorporar más parámetros de entrada en las cadenas, y añadir memoria al proceso para que la información se conserve a lo largo de la secuencia.
Referencia: https://python.langchain.com/docs/modules/chains/foundational/sequential_chains